SAGAN

In [0]:
from keras import backend as K
from keras.layers import Layer
from keras.layers import  InputSpec
from matplotlib import pyplot

Using TensorFlow backend.


In [0]:
class SelfAttention(Layer):

    def __init__(self, ch, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)
        self.channels = ch
    def build(self, input_shape):

        # Create a trainable weight variable for this layer:
        self.gamma = self.add_weight(name='gamma', shape=[1], initializer='zeros', trainable=True) #This is gamma parameter which is learnable parameter, initially set to 0
        self.f = self.add_weight(shape=(1,1,self.channels,self.channels//8),initializer='glorot_uniform',name='kernel_f',trainable=True)
        self.g = self.add_weight(shape=(1,1,self.channels,self.channels//8),initializer='glorot_uniform',name='kernel_g',trainable=True)
        self.h = self.add_weight(shape=(1,1,self.channels,self.channels),initializer='glorot_uniform',name='kernel_h',trainable=True)

        super(SelfAttention, self).build(input_shape)

    def call(self, x):
        def hw_flatten(x):
            #print(K.int_shape(x))
            return K.reshape(x, shape=[K.shape(x)[0], K.shape(x)[1]*K.shape(x)[2], K.shape(x)[3]])

        f = K.conv2d(x,kernel=self.f,strides=(1, 1), padding='same')  
        g = K.conv2d(x,kernel=self.g,strides=(1, 1), padding='same')  
        h = K.conv2d(x,kernel=self.h,strides=(1, 1), padding='same')  
        #print(K.int_shape(f))
        #print(K.int_shape(g))
        #print(K.int_shape(h))
        s = K.batch_dot(hw_flatten(g), K.permute_dimensions(hw_flatten(f), (0, 2, 1))) 

        beta = K.softmax(s, axis=-1)  # attention map
        #pyplot.matshow(K.eval(beta))
        #puplot.show()
        o = K.batch_dot(beta, hw_flatten(h))  

        o = K.reshape(o, shape=K.shape(x))  
        x = self.gamma * o + x

        return x

    def compute_output_shape(self, input_shape):
        return input_shape
